In [6]:
import os
import pandas as pd
import re

folder = "."
arquivos = [f for f in os.listdir(folder) if f.endswith('.xlsx') and not f.startswith('ANOSINICIAIS_curriculo_normalizado')]

linhas = []

def encontrar_coluna(df, padrao):
    for col in df.columns:
        nome = re.sub(r"\s+", " ", col.upper().strip())
        if re.search(padrao, nome):
            return col
    return None

def normalizar_ano(ano_str):
    """Normaliza o formato do ano para 1º Ano, 2º Ano, etc."""
    if not ano_str:
        return ""
    
    # Remove espaços extras e converte para maiúsculo
    ano_clean = re.sub(r"\s+", " ", ano_str.strip().upper())
    
    # Extrai o número
    numero_match = re.search(r"(\d+)", ano_clean)
    if not numero_match:
        return ano_str
    
    numero = numero_match.group(1)
    
    # Mapeia para o formato correto
    mapa_anos = {
        "1": "1º Ano",
        "2": "2º Ano", 
        "3": "3º Ano",
        "4": "4º Ano",
        "5": "5º Ano"
    }
    
    return mapa_anos.get(numero, ano_str)

def normalizar_disciplina(disciplina_str):
    """Normaliza o nome da disciplina com acentos corretos"""
    if not disciplina_str:
        return ""
    
    # Remove espaços extras e converte para maiúsculo
    disciplina_clean = re.sub(r"\s+", " ", disciplina_str.strip().upper())
    
    # Mapeia disciplinas para a forma correta
    mapa_disciplinas = {
        "ARTE": "ARTE",
        "ARTES": "ARTE",
        "MATEMATICA": "MATEMÁTICA",
        "MATEMÁTICA": "MATEMÁTICA",
        "PORTUGUES": "PORTUGUÊS",
        "PORTUGUÊS": "PORTUGUÊS",
        "LINGUA PORTUGUESA": "LÍNGUA PORTUGUESA",
        "LÍNGUA PORTUGUESA": "LÍNGUA PORTUGUESA",
        "CIENCIAS": "CIÊNCIAS",
        "CIÊNCIAS": "CIÊNCIAS",
        "HISTORIA": "HISTÓRIA",
        "HISTÓRIA": "HISTÓRIA",
        "GEOGRAFIA": "GEOGRAFIA",
        "EDUCACAO FISICA": "EDUCAÇÃO FÍSICA",
        "EDUCAÇÃO FISICA": "EDUCAÇÃO FÍSICA",
        "EDUCAÇÃO FÍSICA": "EDUCAÇÃO FÍSICA",
        "ENSINO RELIGIOSO": "ENSINO RELIGIOSO",
        "INGLES": "INGLÊS",
        "INGLÊS": "INGLÊS",
        "LINGUA INGLESA": "LÍNGUA INGLESA",
        "LÍNGUA INGLESA": "LÍNGUA INGLESA"
    }
    
    return mapa_disciplinas.get(disciplina_clean, disciplina_clean)

for arquivo in arquivos:
    caminho = os.path.join(folder, arquivo)
    df_raw = pd.read_excel(caminho, header=None)
    # Extrai o título da linha 2 (índice 1)
    titulo = ""
    for cell in df_raw.iloc[1]:
        if isinstance(cell, str) and ("ANO" in str(cell).upper() or "BIMESTRE" in str(cell).upper()):
            titulo = cell
            break
    
    # Extrai ANO (ex: "2º ANO") e DISCIPLINA (ex: "ARTE")
    ano_match = re.search(r"(\d+[ºª]?\s*ANO)", titulo.upper()) if titulo else None
    
    # Regex dinâmica para diferentes separadores: hífen, travessão, etc.
    disciplina_match = re.search(r"[-–—]\s*([^-–—]+?)\s*[-–—]", titulo) if titulo else None
    
    # Se não encontrou com separadores, tenta extrair a palavra do meio
    if not disciplina_match and titulo:
        # Tenta dividir por espaços e pegar palavras que não sejam número/ano/bimestre
        palavras = titulo.split()
        for palavra in palavras:
            palavra_upper = palavra.upper()
            if (not re.search(r"\d+[ºª]?", palavra_upper) and 
                "ANO" not in palavra_upper and 
                "BIMESTRE" not in palavra_upper and
                len(palavra) > 2):
                disciplina_match = palavra
                break
    
    ano = ano_match.group(1).strip() if ano_match else ""
    
    if isinstance(disciplina_match, str):
        disciplina = disciplina_match.strip().upper()
    elif disciplina_match:
        disciplina = disciplina_match.group(1).strip().upper()
    else:
        disciplina = ""

    # Lê a planilha pulando as 3 primeiras linhas (header na linha 3)
    df = pd.read_excel(caminho, header=2)
    df.columns = [col.strip() for col in df.columns]
    df = df.fillna(method="ffill")  # Preenche células mescladas

    col_habilidades = encontrar_coluna(df, r"HABILIDADES")
    col_orientacoes = encontrar_coluna(df, r"ORIENTA(Ç|C)(Õ|O)ES.*PEDAG(Ó|O)GICAS")
    
    # Debug: mostra as informações extraídas
    print(f"Arquivo: {arquivo}")
    print(f"Título: {titulo}")
    print(f"Ano extraído: '{ano}' -> Normalizado: '{normalizar_ano(ano)}'")
    print(f"Disciplina extraída: '{disciplina}' -> Normalizada: '{normalizar_disciplina(disciplina)}'")
    print(f"Colunas encontradas - Habilidades: {col_habilidades}, Orientações: {col_orientacoes}")
    print("---")

    for _, row in df.iterrows():
        habilidades = str(row[col_habilidades]).replace('\n', ' ').replace('\r', ' ') if col_habilidades else ""
        orientacoes = str(row[col_orientacoes]).replace('\n', ' ').replace('\r', ' ') if col_orientacoes else ""

        # Normaliza ano e disciplina antes de adicionar
        ano_normalizado = normalizar_ano(ano)
        disciplina_normalizada = normalizar_disciplina(disciplina)

        linhas.append({
            "SEGMENTO": "ANOS INICIAIS",
            "ANO": ano_normalizado,
            "DISCIPLINA": disciplina_normalizada,
            "HABILIDADES": habilidades,
            "ORIENTACOES_PEDAGOGICAS": orientacoes
        })

df_final = pd.DataFrame(linhas)
df_final.to_excel("ANOSINICIAIS_curriculo_normalizado.xlsx", index=False)
print("Arquivo ANOSINICIAIS_curriculo_normalizado.xlsx criado com sucesso!")

C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas


Arquivo: 1º ano - ARTE - 1º Bimestre.xlsx
Título: 1º ano - ARTE - 1º Bimestre 
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'ARTE' -> Normalizada: 'ARTE'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 1º ANO - ARTE - 2º BIMESTRE.xlsx
Título:  1º ANO - ARTE - 2º BIMESTRE 
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'ARTE' -> Normalizada: 'ARTE'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 1º ANO - ARTE - 3º BIMESTRE.xlsx
Título: 1º ANO - ARTE - 3º BIMESTRE
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'ARTE' -> Normalizada: 'ARTE'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---


C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future vers

Arquivo: 1º ANO - ARTE - 4 º BIMESTRE.xlsx
Título: 1º ANO - ARTE - 4 º BIMESTRE
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'ARTE' -> Normalizada: 'ARTE'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 1º ANO - CIENCIAS - 2º BIMESTRE.xlsx
Título: 1º ANO - CIENCIAS - 2º BIMESTRE
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'CIENCIAS' -> Normalizada: 'CIÊNCIAS'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 1º ANO - CIENCIAS - 3º BIMESTRE.xlsx
Título: 1º ANO - CIENCIAS - 3º BIMESTRE
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'CIENCIAS' -> Normalizada: 'CIÊNCIAS'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 1º ANO - CIENCIAS - 4º BIMESTRE.xlsx
Título: 1º ANO - CIENCIAS - 4º BIMESTRE
Ano extraído: '1º ANO' -> Normalizado: '1º Ano'
Disciplina extraída: 'CIENCIAS' ->

C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future vers

...

C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas


Arquivo: 5º ANO - LINGUA PORTUGUESA - 2º BIMESTRE.xlsx
Título: 5º ANO - LINGUA PORTUGUESA - 2º BIMESTRE
Ano extraído: '5º ANO' -> Normalizado: '5º Ano'
Disciplina extraída: 'LINGUA PORTUGUESA' -> Normalizada: 'LÍNGUA PORTUGUESA'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---


C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future vers

Arquivo: 5º ANO - LINGUA PORTUGUESA - 3º BIMESTRE.xlsx
Título: 5º ANO - LINGUA PORTUGUESA - 3º BIMESTRE
Ano extraído: '5º ANO' -> Normalizado: '5º Ano'
Disciplina extraída: 'LINGUA PORTUGUESA' -> Normalizada: 'LÍNGUA PORTUGUESA'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 5º ANO – CIÊNCIAS - 1º BIMESTRE.xlsx
Título: 5º ANO – CIÊNCIAS - 1º BIMESTRE
Ano extraído: '5º ANO' -> Normalizado: '5º Ano'
Disciplina extraída: 'CIÊNCIAS' -> Normalizada: 'CIÊNCIAS'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 5º ANO – MATEMÁTICA  - 1º Bimestre.xlsx
Título: 5º ANO – MATEMÁTICA  - 1º Bimestre
Ano extraído: '5º ANO' -> Normalizado: '5º Ano'
Disciplina extraída: 'MATEMÁTICA' -> Normalizada: 'MATEMÁTICA'
Colunas encontradas - Habilidades: HABILIDADES, Orientações: ORIENTAÇÕES PEDAGÓGICAS
---
Arquivo: 5º ANO – MATEMÁTICA - 2º Bimestre.xlsx
Título: 5º ANO – MATEMÁTICA - 2º Bimestre
Ano extraído: '5º 

C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
C:\Users\guilherme.saito\AppData\Local\Temp\ipykernel_8848\2009880055.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")  # Preenche células mescladas
